# Support Vector Machine (SVM) Classification on raw dataset

Link to Excel file of extracted features: https://drive.google.com/drive/u/1/folders/1ZMbW7HxNkBUAlnT-_lXLD4J_r0A778SJ

In [1]:
# import required packages
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV


In [2]:
# read in features csv file
df = pd.read_csv('ck_features.csv')
df

,Total Power,Abs LFP,Abs HFP,nLFP,nHFP,LF/HF,RMSSD,SDNN,label
0,11961.755298,3249.551607,8043.922163,0.271662,0.672470,0.403976,932.714731,54.825976,ck_pre
1,9211.352702,2022.717898,6829.397915,0.219590,0.741411,0.296178,803.437128,120.850089,ck_pre
2,9283.913716,2378.753676,6678.590063,0.256223,0.719372,0.356176,793.228617,162.223603,ck_pre
3,9994.154159,2431.402950,7008.622545,0.243283,0.701272,0.346916,865.589904,67.238339,ck_pre
4,9597.894148,1897.800009,7150.309660,0.197731,0.744987,0.265415,852.451001,70.600140,ck_pre
...,...,...,...,...,...,...,...,...,...
283,1160.095357,223.552939,813.610805,0.192702,0.701331,0.274766,735.611693,147.849426,ck_med
284,1420.536153,275.481168,1015.442748,0.193928,0.714831,0.271292,766.606858,156.710024,ck_med
285,1393.953672,196.719824,1092.517596,0.141124,0.783755,0.180061,779.975532,148.275939,ck_med
286,1267.482134,253.493676,906.193736,0.199998,0.714956,0.279735,760.283782,139.385959,ck_med


In [3]:
# encode meditative/non-meditative labels
categories = {'ck_pre':0,'ck_med':1}
df['label'] = df['label'].replace(categories)
df

,Total Power,Abs LFP,Abs HFP,nLFP,nHFP,LF/HF,RMSSD,SDNN,label
0,11961.755298,3249.551607,8043.922163,0.271662,0.672470,0.403976,932.714731,54.825976,0
1,9211.352702,2022.717898,6829.397915,0.219590,0.741411,0.296178,803.437128,120.850089,0
2,9283.913716,2378.753676,6678.590063,0.256223,0.719372,0.356176,793.228617,162.223603,0
3,9994.154159,2431.402950,7008.622545,0.243283,0.701272,0.346916,865.589904,67.238339,0
4,9597.894148,1897.800009,7150.309660,0.197731,0.744987,0.265415,852.451001,70.600140,0
...,...,...,...,...,...,...,...,...,...
283,1160.095357,223.552939,813.610805,0.192702,0.701331,0.274766,735.611693,147.849426,1
284,1420.536153,275.481168,1015.442748,0.193928,0.714831,0.271292,766.606858,156.710024,1
285,1393.953672,196.719824,1092.517596,0.141124,0.783755,0.180061,779.975532,148.275939,1
286,1267.482134,253.493676,906.193736,0.199998,0.714956,0.279735,760.283782,139.385959,1


In [4]:
# split dependent and independent variables
# target variable
Y = df['label'].values
# features
X = df.drop(labels = ['label'], axis = 1)
features_list = list(X.columns)

# Standardizing the features
X = StandardScaler().fit_transform(X)

In [5]:
# split training and testing dataset: 70% training, 30% testing
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.3,random_state = 42)

## Linear SVM

In [6]:
# create lists of possible parameters for linear SVM
# to be used in grid search to find the best possible parameters later

lin_clf_loss = ['hinge', 'squared_hinge']
lin_clf_dual = [True,False]
lin_clf_tol = [0.001,0.0001,0.00001]

# Create the grid
lin_grid = {'loss': lin_clf_loss,
               'dual': lin_clf_dual,
               'tol': lin_clf_tol}

lin_grid

{'loss': ['hinge', 'squared_hinge'],
 'dual': [True, False],
 'tol': [0.001, 0.0001, 1e-05]}

In [7]:
# run parameters through model and find best set of parameters
# this step splits the training data into mini train/test datasets to find the best hyperparameters for the model

import time
print(time.ctime(time.time()))

# Create the model to be tuned
lin_base = svm.LinearSVC()

# Create the random search to find best hyperparameters
lin_random = RandomizedSearchCV(estimator = lin_base, 
                                param_distributions = lin_grid, 
                                n_iter = 20,
                                cv = 10, 
                                verbose = 2, 
                                random_state = 42, 
                                n_jobs = -1)

# Fit the random search model on training data
lin_random.fit(X_train, y_train)

print(time.ctime(time.time()))

Wed Apr 14 19:24:57 2021
Fitting 10 folds for each of 12 candidates, totalling 120 fits


/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 12 is smaller than n_iter=20. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.3s


Wed Apr 14 19:25:01 2021


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    4.4s finished


In [8]:
# show best set of parameters
lin_random.best_params_

{'tol': 0.001, 'loss': 'squared_hinge', 'dual': True}

In [6]:
# Create the linear SVM model
lin_model = svm.LinearSVC(tol = 0.001,
                      loss = 'squared_hinge',
                      dual = True,
                      random_state = 42)

In [7]:
# fit model
lin_model.fit(X_train,y_train)

LinearSVC(random_state=42, tol=0.001)

In [8]:
# predict y values from testing data
y_pred = lin_model.predict(X_test)

In [9]:
# display results in confusion matrix
cm = confusion_matrix(y_test,y_pred)
cm

results = pd.crosstab(y_test, y_pred, colnames=['Predicted'])
results = results.rename(columns={0: 'Non-Meditative', 1: 'Meditative'})
results.index = ['Non-Meditative', 'Meditative']
index = results.index
index.name = "Actual"
results

Predicted,Non-Meditative,Meditative
Actual,,
Non-Meditative,37,7
Meditative,9,34


In [10]:
target_names = ['Non-Meditative', 'Meditative']
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

Non-Meditative       0.80      0.84      0.82        44
    Meditative       0.83      0.79      0.81        43

      accuracy                           0.82        87
     macro avg       0.82      0.82      0.82        87
  weighted avg       0.82      0.82      0.82        87



In [11]:
# store metrics for comparision amongst SVM kernels later
svm_rec = []
svm_acc = []
svm_pre = []
svm_mcc = []

# obtain metrics for model
recall = metrics.recall_score(y_test,y_pred)
accuracy = metrics.accuracy_score(y_test,y_pred)
precision = metrics.precision_score(y_test,y_pred)
mcc = metrics.matthews_corrcoef(y_test,y_pred)

svm_rec.append(recall)
svm_acc.append(accuracy)
svm_pre.append(precision)
svm_mcc.append(mcc)

# display metrics
print('Linear SVM\n')
print('Sensitivity = %.3f' % recall)
print('Accuracy = %.3f' % accuracy)
print('Precision = %.3f' % precision)
print('Matthew Correlation Coefficient = %.3f' % mcc)


Linear SVM

Sensitivity = 0.791
Accuracy = 0.816
Precision = 0.829
Matthew Correlation Coefficient = 0.633


In [12]:
# show most important features when determining classification category
feature_imp = pd.Series(lin_model.coef_[0], index = features_list).sort_values(ascending = False)
feature_imp

Abs LFP        1.209159
SDNN           0.328190
Total Power    0.302592
RMSSD         -0.353894
LF/HF         -0.446822
Abs HFP       -0.842119
nHFP          -1.254099
nLFP          -1.986256
dtype: float64

## Radial Basis Function SVM

In [38]:
# create lists of possible parameters for rbf SVM
# to be used in grid search to find the best possible parameters later

rbf_gamma = [0.1, 1,'scale','auto']
rbf_tol = [0.001,0.0001,0.00001]

# Create the grid
rbf_grid = {'gamma': rbf_gamma,
               'tol': rbf_tol}

rbf_grid


{'gamma': [0.1, 1, 'scale', 'auto'], 'tol': [0.001, 0.0001, 1e-05]}

In [39]:
# run parameters and find set with best score
# this step splits the training data into mini train/test datasets to find the best hyperparameters for the model

print(time.ctime(time.time()))

# Create the model to be tuned
rbf_base = svm.SVC(kernel = 'rbf')

# Create the random search to find best hyperparameters
rbf_random = RandomizedSearchCV(estimator = rbf_base, 
                                param_distributions = rbf_grid, 
                                n_iter = 10,
                                cv = 10, 
                                verbose = 2, 
                                random_state = 42, 
                                n_jobs = -1)

# Fit the random search model on training data
rbf_random.fit(X_train, y_train)

print(time.ctime(time.time()))

Wed Apr 14 19:45:50 2021
Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  93 tasks      | elapsed:    0.4s


Wed Apr 14 19:45:51 2021


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.8s finished


In [40]:
# display best rbf svm parameters
rbf_random.best_params_

{'tol': 1e-05, 'gamma': 1}

In [13]:
# Create the rbf SVM model
rbf_model = svm.SVC(kernel = 'rbf',
                    tol = 0.001,
                      gamma = 1,
                      random_state = 42)

# fit model
rbf_model.fit(X_train,y_train)

# predict testing data
y_pred = rbf_model.predict(X_test)

# display confusion matrix
results = pd.crosstab(y_test, y_pred, colnames=['Predicted'])
results = results.rename(columns={0: 'Non-Meditative', 1: 'Meditative'})
results.index = ['Non-Meditative', 'Meditative']
index = results.index
index.name = "Actual"
results

Predicted,Non-Meditative,Meditative
Actual,,
Non-Meditative,37,7
Meditative,4,39


In [14]:
# obtain metrics of rbf model
recall = metrics.recall_score(y_test,y_pred)
accuracy = metrics.accuracy_score(y_test,y_pred)
precision = metrics.precision_score(y_test,y_pred)
mcc = metrics.matthews_corrcoef(y_test,y_pred)

svm_rec.append(recall)
svm_acc.append(accuracy)
svm_pre.append(precision)
svm_mcc.append(mcc)

print('Radial Basic Function SVM\n')
print('Sensitivity = %.3f' % recall)
print('Accuracy = %.3f' % accuracy)
print('Precision = %.3f' % precision)
print('Matthew Correlation Coefficient = %.3f' % mcc)

Radial Basic Function SVM

Sensitivity = 0.907
Accuracy = 0.874
Precision = 0.848
Matthew Correlation Coefficient = 0.749


## Polynomial SVM

In [46]:
# create lists of possible parameters for polynomial SVM
# to be used in grid search to find the best possible parameters later

poly_degree = [2,3,4,5,6,7]
poly_gamma = [0.1, 1,'scale','auto']
poly_tol = [0.001,0.0001,0.00001]

# Create the grid
poly_grid = {'gamma': poly_gamma,
            'degree': poly_degree,
               'tol': poly_tol}

poly_grid


{'gamma': [0.1, 1, 'scale', 'auto'],
 'degree': [2, 3, 4, 5, 6, 7],
 'tol': [0.001, 0.0001, 1e-05]}

In [48]:
# this step splits the training data into mini train/test datasets to find the best hyperparameters for the model

print(time.ctime(time.time()))

# Create the model to be tuned
poly_base = svm.SVC(kernel = 'poly')

# Create the random search to find best hyperparameters
poly_random = RandomizedSearchCV(estimator = poly_base, 
                                param_distributions = poly_grid, 
                                n_iter = 100,
                                cv = 10, 
                                verbose = 2, 
                                random_state = 42, 
                                n_jobs = -1)

# Fit the random search model on training data
poly_random.fit(X_train, y_train)

print(time.ctime(time.time()))

Wed Apr 14 19:51:06 2021
Fitting 10 folds for each of 72 candidates, totalling 720 fits


/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 72 is smaller than n_iter=100. Running 72 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  92 tasks      | elapsed:    1.2s


Wed Apr 14 19:51:12 2021


[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:    6.5s finished


In [49]:
# display best polynomial svm parameters
poly_random.best_params_

{'tol': 0.001, 'gamma': 1, 'degree': 3}

In [15]:
# Create the poly SVM model
poly_model = svm.SVC(kernel = 'poly',
                    tol = 0.001,
                    gamma = 1,
                    degree = 3,
                    random_state = 42)

# fit model
poly_model.fit(X_train,y_train)

# predict test data
y_pred = poly_model.predict(X_test)

# confusion matrix
results = pd.crosstab(y_test, y_pred, colnames=['Predicted'])
results = results.rename(columns={0: 'Non-Meditative', 1: 'Meditative'})
results.index = ['Non-Meditative', 'Meditative']
index = results.index
index.name = "Actual"
results

Predicted,Non-Meditative,Meditative
Actual,,
Non-Meditative,40,4
Meditative,6,37


In [16]:
# evaluation metrics of poly svm model
recall = metrics.recall_score(y_test,y_pred)
accuracy = metrics.accuracy_score(y_test,y_pred)
precision = metrics.precision_score(y_test,y_pred)
mcc = metrics.matthews_corrcoef(y_test,y_pred)

svm_rec.append(recall)
svm_acc.append(accuracy)
svm_pre.append(precision)
svm_mcc.append(mcc)

print('Polynomial SVM\n')
print('Sensitivity = %.3f' % recall)
print('Accuracy = %.3f' % accuracy)
print('Precision = %.3f' % precision)
print('Matthew Correlation Coefficient = %.3f' % mcc)

Polynomial SVM

Sensitivity = 0.860
Accuracy = 0.885
Precision = 0.902
Matthew Correlation Coefficient = 0.771


### Comparison Between SVM models

In [20]:
df = pd.DataFrame()

df['svm kernel'] = ['linear','rbf','poly']
df['sensitivity'] = svm_rec
df['accuracy'] = svm_acc
df['precision'] = svm_pre
df['mcc'] = svm_mcc

df

,svm kernel,sensitivity,accuracy,precision,mcc
0,linear,0.790698,0.816092,0.829268,0.632611
1,rbf,0.906977,0.873563,0.847826,0.749075
2,poly,0.860465,0.885057,0.902439,0.770779


In [58]:
y_axis_labels = ['linear','rbf','polynomial']

plt.figure(dpi = 120)
ax = sns.heatmap(df.iloc[:, 1:],
                 cmap = 'GnBu',
                 annot = True,
                 yticklabels = y_axis_labels,
                )

plt.yticks(rotation=0) 
plt.show()

Polynomial SVM showed best performance overall